In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
# import functions from the modsim.py module
from modsim import *
import scipy.signal as sigw

In [2]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
A = UNITS.ampere
V = UNITS.volts
Ohm = UNITS.ohm
N = UNITS.newton
U_mu = N / A**2

<Unit('newton / ampere ** 2')>

In [3]:
params = Params(x = 0 * m, 
                d = 1 * m,
                r = 0.1 * m,
                L = 100 * m,
                mass = 1 * kg,
                Vs = 1000 * V,
                R = 0.01 * Ohm,
                mu_0 = 1.257e-6 * U_mu,
                t_end = 100 * s)

,values
x,0 meter
d,1 meter
r,0.1 meter
L,100 meter
mass,1 kilogram
Vs,1000 volt
R,0.01 ohm
mu_0,1.257e-06 newton / ampere ** 2
t_end,100 second


In [4]:
def make_system(params):
    """
    Make a system object.
    
    params: Params object containing following:
        x: Initial position of armature
        d: Separation distance between two rails
        r: Radii of the rails
        L: Length armature has to travel along rails before launch
        mass: Mass of armature
        Vs: (Initial) Voltage supply magnitude
        R: Resistance of railgun circuit
        mu_0: Vacuum permeability constant
        t_end: Simulation end time
               
    returns: System object containing
        params values
        c_mag: Calculated magnetic force coefficient
        init: Initial state object containing:
            x: Position of armature (initialized to params.x)
            v: Velocity of armature (initialized to 0)
    """
    
    # make the initial state
    init = State(x=params.x, v=0)
    
    # Determine magnetic force coefficient
    c_mag = params.mu_0 * log((params.d - params.r) / params.r) / (2 * pi)
    
    return System(params, c_mag = c_mag, init=init)

In [5]:
def mag_force(I, system):
    return system.c_mag * I**2

In [6]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, v = state

    I = system.Vs / system.R
    
    F_mag = mag_force(I, system)
    a = F_mag / system.mass
    
    return v, a

In [7]:
def event_func(state, t, system):
    """Stop when the y coordinate is 0.
    
    state: State object
    t: time
    system: System object
    
    returns: y coordinate
    """
    x, v = state
    return x - system.L

In [8]:
def range_func(mass, params):  
    """Computes range for a given launch angle.
    
    angle: launch angle in degrees
    params: Params object
    
    returns: distance in meters
    """
    params = Params(params, mass=mass)
    system = make_system(params)
    results, details = run_ode_solver(system, slope_func, events=event_func)
    return results.x, results.v

In [9]:
x, v = range_func(1, params)

In [10]:
x

,values
0.000000,0.000000
0.000100,0.021979
0.001100,2.659410
0.006745,100.000000
